In [ ]:
import sys
import numpy as np
import pandas as pd
import yaml
from pathlib import Path
# Allow imports from project scripts
#sys.path.append(str(Path.cwd().parents[0]))
project_root = Path.cwd().parents[0]
sys.path.append(str(project_root))
from scripts.paths import PROJECT_ROOT, load_config

with open("../configs/sim1.yaml", 'r') as f:
    config = yaml.safe_load(f)
sim_cfg = config["simulation"]

sim_cfg

In [ ]:
u = 931.5 # atomic mass unit 

def kinetic_energy(A, beta):
    gamma = 1 / np.sqrt(1 - beta**2)
    return (gamma - 1) * A * u

def beta(A, E_kin):
    gamma = E_kin / (A * u) + 1
    beta = np.sqrt(1 - 1 / gamma**2)
    return beta


In [ ]:
def simulate_event(Z, A, b_in, layers, reaction_prob, layer_thickness, delta_Z=1, eloss_scaling = 1.0):
    """
    Simulate one particle through a layered detector with potential intra-layer reaction.
    Reaction changes Z and A but keeps velocity constant at the moment of reaction.
    """
    b = b_in
    current_Z = Z
    current_A = A
    E_losses = []
    reacted = False
    reaction_layer = 0  # 0 means no reaction
    reaction_depth = None
    
    for i in range(layers):
        thickness = layer_thickness[i]
        reaction_chance = reaction_prob * (thickness / np.mean(layer_thickness))
        if not reacted and np.random.rand() < reaction_chance:
            reacted = True
            reaction_layer = i + 1
            reaction_depth = np.random.uniform(0, thickness)
            
            # Pre-reaction energy loss
            sp_pre = (current_Z ** 2) / (b ** 2) * eloss_scaling
            dE_pre = sp_pre * reaction_depth
            
            E_kin_pre = kinetic_energy(current_A, b)
            E_kin_post = max(E_kin_pre - dE_pre, 1e-6)
            b_post = beta(current_A, E_kin_post)
            
            # Apply reaction (Z and A change, velocity remains same for this layer)
            current_Z = max(current_Z - delta_Z, 1)
            current_A = max(current_A - delta_Z, 1)

            # Post-reaction energy loss
            d2 = thickness - reaction_depth
            sp_post = (current_Z ** 2) / (b_post ** 2) * eloss_scaling
            dE_post = sp_post * d2
            b = beta(current_A, max(E_kin_post - dE_post, 1e-6))

            delta_E = dE_pre + dE_post
            
        else:
            sp = (current_Z ** 2) / (b ** 2) * eloss_scaling
            delta_E = sp * thickness
            
            E_kin = kinetic_energy(current_A, b)
            E_kin = max(E_kin - delta_E, 1e-6)
            b = beta(current_A, E_kin)

        E_losses.append(delta_E)

        # Update kinetic energy and velocity after the whole layer
        E_kin = kinetic_energy(current_A, b)
        E_kin = max(E_kin - delta_E, 1e-6)
        b = beta(current_A, E_kin)

    return {
        'b_in': b_in,
        'b_out': b,
        'reaction_layer': reaction_layer,
        'reaction_depth': reaction_depth,
        **{f'dE_{i+1}': E_losses[i] for i in range(layers)}
    }

In [ ]:
def generate_dataset(n, velocity_distribution="uniform", b_min=None, b_max=None, b_mean=None, b_sigma=None, **kwargs):
    """
    Generate a dataset of events using simulate_event.
    Handles velocity distribution setup.
    """
    if velocity_distribution == "gaussian":
        assert b_mean is not None and b_sigma is not None, "b_mean and b_sigma must be set for gaussian distribution"
        betas = np.random.normal(loc=b_mean, scale=b_sigma, size=n)
        betas = np.clip(betas, 0, 1)
    else:
        assert b_min is not None and b_max is not None, "b_min and b_max must be set for uniform distribution"
        betas = np.random.uniform(low=b_min, high=b_max, size=n)

    return pd.DataFrame([
        simulate_event(b_in=b, **kwargs) for b in tqdm(betas, desc="Simulating events")
    ])


In [ ]:
from tqdm.notebook import tqdm  # For nice Jupyter-style progress bars
# Generate the dataset
df = generate_dataset(
    n=sim_cfg["n_events"],
    velocity_distribution=sim_cfg.get("velocity_distribution", "uniform"),
    b_min=sim_cfg.get("b_min"),
    b_max=sim_cfg.get("b_max"),
    b_mean=sim_cfg.get("b_mean"),
    b_sigma=sim_cfg.get("b_sigma"),
    Z=sim_cfg["Z"],
    A=sim_cfg["A"],
    delta_Z=sim_cfg["delta_Z"],
    layers=sim_cfg["layers"],
    eloss_scaling=sim_cfg["eloss_scaling"],
    reaction_prob=sim_cfg["reaction_prob"],
    layer_thickness=sim_cfg["layer_thickness"]
)

# Show a sample
df.head()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

def plot_energy_distribution_3d(df, bins=50, by_reaction=False, max_layer=5):
    layers = [f'dE_{i+1}' for i in range(max_layer)]

    if by_reaction:
        reaction_layers = sorted(df['reaction_layer'].unique())
    else:
        reaction_layers = [None]

    for rl in reaction_layers:
        fig = plt.figure(figsize=(12, 6))
        ax = fig.add_subplot(111, projection='3d')

        data = df if rl is None else df[df['reaction_layer'] == rl]
        label = "All events" if rl is None else f"reaction_layer = {rl}"

        for i, col in enumerate(layers):
            counts, edges = np.histogram(data[col], bins=bins)
            xpos = (edges[:-1] + edges[1:]) / 2
            ypos = np.full_like(xpos, i + 1)
            zpos = np.zeros_like(xpos)
            dx = (edges[1] - edges[0]) * 0.9
            dy = 0.6
            dz = counts

            ax.bar3d(xpos, ypos, zpos, dx, dy, dz, alpha=0.7)

        ax.set_xlabel("Energy Loss [MeV]")
        ax.set_ylabel("Layer")
        ax.set_zlabel("Count")
        ax.set_title(f"Energy Loss Distribution per Layer ({label})")
        plt.tight_layout()
        plt.show()

In [ ]:
plot_energy_distribution_3d(df, bins=40, by_reaction=False)

In [ ]:
import matplotlib.pyplot as plt

layers = sim_cfg["layers"]
thicknesses = sim_cfg["layer_thickness"]
reaction_layers = df["reaction_layer"]
reaction_depths = df["reaction_depth"]

# Filter for only reacted events
reacted_df = df[df["reaction_layer"] > 0]

# --- FIXED: height_ratios must go in gridspec_kw
fig, (ax1, ax2) = plt.subplots(
    2, 1, figsize=(8, 6),
    sharex=False,
    gridspec_kw={"height_ratios": [1, 1.2]}
)

# Panel 1: Reaction layer histogram
ax1.hist(reaction_layers, bins=np.arange(-0.5, layers + 1.5), edgecolor='black')
ax1.set_title("Distribution of Reaction Layers")
ax1.set_ylabel("Count")
ax1.set_xticks(range(layers + 1))

# Panel 2: Reaction depth per layer
colors = plt.cm.viridis(np.linspace(0, 1, layers))
for i in range(1, layers + 1):
    sub = reacted_df[reacted_df["reaction_layer"] == i]
    if not sub.empty:
        ax2.hist(sub["reaction_depth"], bins=30, alpha=0.6, color=colors[i - 1],
                 label=f"Layer {i} ({thicknesses[i-1]} mm)", histtype='stepfilled')

ax2.set_title("Distribution of Reaction Depths (within layer)")
ax2.set_xlabel("Depth into Layer [mm]")
ax2.set_ylabel("Count")
ax2.legend(loc="upper right", fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
from pandas.plotting import scatter_matrix

from itertools import combinations
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

cols = [f'dE_{i+1}' for i in range(sim_cfg["layers"])]
num_vars = len(cols)

# Normalize reaction_layer to colormap
norm = Normalize(vmin=df["reaction_layer"].min(), vmax=df["reaction_layer"].max())
cmap = plt.cm.viridis
reaction_layer_values = df["reaction_layer"]
colors = cmap(norm(reaction_layer_values))

fig, axes = plt.subplots(num_vars, num_vars, figsize=(12, 12))
fig.suptitle("Pairwise Energy Loss (Colored by Reaction Layer)", fontsize=16, y=1.02)

for i in range(num_vars):
    for j in range(num_vars):
        ax = axes[i, j]
        ax.grid(False)

        if i == j:
            # Diagonal: color-coded histograms
            for layer_val in sorted(df["reaction_layer"].unique()):
                subset = df[df["reaction_layer"] == layer_val][cols[i]]
                ax.hist(subset, bins=30, alpha=0.5,
                        color=cmap(norm(layer_val)), label=f"Layer {layer_val}")
        else:
            ax.scatter(df[cols[j]], df[cols[i]], c=colors, s=10, alpha=0.6)

        if i == num_vars - 1:
            ax.set_xlabel(cols[j])
        else:
            ax.set_xticks([])

        if j == 0:
            ax.set_ylabel(cols[i])
        else:
            ax.set_yticks([])

# Adjust plot to fit colorbar on the right
fig.subplots_adjust(top=0.92, wspace=0.1, hspace=0.1, right=0.88)

# Add external colorbar
sm = ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
cbar_ax = fig.add_axes([0.9, 0.3, 0.02, 0.4])
cbar_ax.grid(False)  
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label("Reaction Layer")

plt.show()

In [ ]:
from scipy.stats import gaussian_kde

plt.figure(figsize=(10, 6))

for layer in range(sim_cfg["layers"]):
    col = f'dE_{layer+1}'
    
    if col not in df.columns:
        print(f"Skipping {col}: not found")
        continue

    data = df[col].dropna().astype(float).to_numpy()

    if len(np.unique(data)) < 2:
        print(f"Skipping {col}: not enough variation")
        continue

    kde = gaussian_kde(data)
    x_vals = np.linspace(min(data), max(data), 200)
    plt.plot(x_vals, kde(x_vals), label=col)

plt.title("Distribution of Energy Loss per Layer")
plt.xlabel("Energy Loss [MeV]")
plt.ylabel("Density")
plt.legend()
plt.tight_layout()
plt.show()
